In [1]:
import cvxopt

In [2]:
from cvxopt import msk

In [5]:
import cvxpy as cp
import numpy as np



The optimal value is -15.220912604449774
A solution x is
[-1.09462326 -0.20466376 -0.80667962  0.07890685  0.49481543 -1.0047415
  1.19966687  0.97398692  0.54537495  0.90952462]
A dual solution is
[1.27313592e-13 6.11756414e-01 5.28171752e-01 1.07296862e+00
 5.97228032e-13 2.30153870e+00 2.13913306e-13 7.61206901e-01
 8.20205732e-12 2.49370375e-01 1.40387338e-13 2.06014071e+00
 3.22417204e-01 3.84054355e-01 3.10002354e-12]


In [62]:
# %%timeit

# Generate a random non-trivial linear program.
m = 15
n = 10
np.random.seed(1)
s0 = np.random.randn(m)
lamb0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)

A = np.random.randn(m, n)
b = A @ x0 + s0
c = - A.T @ lamb0

# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(c.T@x),
                 [A@x <= b])
prob.solve()

# Print result.
# print("\nThe optimal value is", prob.value)
# print("A solution x is")
# print(x.value)
# print("A dual solution is")
# print(prob.constraints[0].dual_value)

-15.22091260446784

In [67]:
x.value

array([-1.10131657, -0.16370661, -0.89711643,  0.03228612,  0.60662428,
       -1.12655967,  1.12985839,  0.88200333,  0.49089264,  0.89851057])

In [69]:
np.asarray([1, 2, 3]).T @ np.asarray([1, 1, 1])

6

In [61]:
prob.unpack()

TypeError: unpack() missing 1 required positional argument: 'solution'

In [12]:
c.shape

(10,)